In [13]:
import numpy as np
import pandas as pd
import datetime
import os

In [425]:
def process(file):
    
    temp = pd.read_csv(file)
    temp = temp[temp['Country/Region'] == 'US'] if 'Country/Region' in temp.columns else temp[temp['Country_Region'] == 'US']
    
    if 'Admin2' in temp.columns:
        temp['County'] = temp.Admin2
        temp['State'] = temp.Province_State
    elif ',' in 'Province/State':
        x = temp['Province/State'].map(lambda x: x.replace(' (From Diamond Princess)','').replace('Unassigned Location','').split(','))
        temp['County'] = x[0]
        temp['State'] = x[-1]
    else:
        temp['County'] = ''
        temp['State'] = temp['Province/State'].map(lambda x: x.replace(' (From Diamond Princess)','').replace('Unassigned Location',''))

    if 'Last Update' in temp.columns:
        temp['Date'] = pd.to_datetime(temp['Last Update']).map(lambda x: x.strftime('%Y-%m-%d'))
    else:
        temp['Date'] = pd.to_datetime(temp['Last_Update']).map(lambda x: x.strftime('%Y-%m-%d'))
    
    temp['Lat'] = temp['Latitude'] if 'Latitude' in temp.columns else np.nan
    
    if 'Longitude' in temp.columns:
        temp['Long'] = temp['Longitude']
    elif 'Long_' in temp.columns:
        temp['Long'] = temp['Long_']
    else:
        temp['Long'] = np.nan
        
    temp['Recovered'] = temp['Recovered'] if 'Recovered' in temp.columns else 0
    temp['Active'] = temp['Active'] if 'Active' in temp.columns else 0
    temp['FIPS'] = temp['FIPS'] if 'FIPS' in temp.columns else 0
    
    temp = temp[['FIPS', 'County', 'State', 'Date', 'Lat', 'Long', 'Confirmed', 'Deaths', 'Recovered', 'Active']]        

    return temp

In [426]:
path = '../../Data/covid_19_daily_reports/'
files = [i for i in os.listdir(path) if 'csv' in i]

df = pd.DataFrame([])

for i in files:
    print('loading {}'.format(i))
    temp = process(path + i)
    df = pd.concat([df, temp], ignore_index=True)
        
print(df.shape)
# data.to_csv(path + 'covid_19_US.csv')

loading 01-22-2020.csv
loading 01-23-2020.csv
loading 01-24-2020.csv
loading 01-25-2020.csv
loading 01-26-2020.csv
loading 01-27-2020.csv
loading 01-28-2020.csv
loading 01-29-2020.csv
loading 01-30-2020.csv
loading 01-31-2020.csv
loading 02-01-2020.csv
loading 02-02-2020.csv
loading 02-03-2020.csv
loading 02-04-2020.csv
loading 02-05-2020.csv
loading 02-06-2020.csv
loading 02-07-2020.csv
loading 02-08-2020.csv
loading 02-09-2020.csv
loading 02-10-2020.csv
loading 02-11-2020.csv
loading 02-12-2020.csv
loading 02-13-2020.csv
loading 02-14-2020.csv
loading 02-15-2020.csv
loading 02-16-2020.csv
loading 02-17-2020.csv
loading 02-18-2020.csv
loading 02-19-2020.csv
loading 02-20-2020.csv
loading 02-21-2020.csv
loading 02-22-2020.csv
loading 02-23-2020.csv
loading 02-24-2020.csv
loading 02-25-2020.csv
loading 02-26-2020.csv
loading 02-27-2020.csv
loading 02-28-2020.csv
loading 02-29-2020.csv
loading 03-01-2020.csv
loading 03-02-2020.csv
loading 03-03-2020.csv
loading 03-04-2020.csv
loading 03-

In [427]:
df.head()

,FIPS,County,State,Date,Lat,Long,Confirmed,Deaths,Recovered,Active
0,0.0,,Washington,2020-01-22,NaN,NaN,1.0,NaN,NaN,0
1,0.0,,Washington,2020-01-23,NaN,NaN,1.0,NaN,NaN,0
2,0.0,,Washington,2020-01-24,NaN,NaN,1.0,NaN,NaN,0
3,0.0,,Chicago,2020-01-24,NaN,NaN,1.0,NaN,NaN,0
4,0.0,,Washington,2020-01-25,NaN,NaN,1.0,NaN,NaN,0
